In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

def load_data_from_folders(folder_path, chunk_size=60):
    X = []
    y = []
    class_names = []
    
    for class_name in os.listdir(folder_path):
        class_folder = os.path.join(folder_path, class_name)
        if os.path.isdir(class_folder):
            class_names.append(class_name)  
            for filename in os.listdir(class_folder):
                if filename.endswith('.npy'):
                    file_path = os.path.join(class_folder, filename)
                    data = np.load(file_path)
                    if len(data) == chunk_size:
                        X.append(data)
                        y.append(class_names.index(class_name))  
                    else:
                        print(f"Skipping {filename}, it doesn't have {chunk_size} data points")
    
    print("Shape of X:", np.array(X).shape)
    print("Shape of y:", np.array(y).shape)
    print("Class names:", class_names)

    return np.array(X), np.array(y), class_names

def create_lstm_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(128, kernel_size=5, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    initial_learning_rate = 1e-3
    optimizer = Adam(learning_rate=initial_learning_rate)
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

folder_path = input("Please enter the path to the folder containing the class folders: ")
X, y, class_names = load_data_from_folders(folder_path)

if X.size == 0 or y.size == 0:
    raise ValueError("No valid data loaded")


scaler = StandardScaler()
X = X.reshape(-1, X.shape[1])  
X = scaler.fit_transform(X)
X = X.reshape((X.shape[0], X.shape[1], 1)) 


y = to_categorical(y, num_classes=len(class_names))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


class_weights = class_weight.compute_class_weight(
    class_weight='balanced', 
    classes=np.unique(np.argmax(y_train, axis=1)),  
    y=np.argmax(y_train, axis=1)  
)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}


input_shape = (X_train.shape[1], X_train.shape[2])  
num_classes = len(class_names)
model = create_lstm_model(input_shape, num_classes)
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

history = model.fit(
    X_train, y_train, 
    epochs=200, 
    batch_size=16, 
    validation_data=(X_test, y_test), 
    class_weight=class_weight_dict,
    callbacks=[early_stopping, lr_scheduler]
)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)


print(classification_report(y_true_classes, y_pred_classes, target_names=class_names))


model.save('lstm_stress_beta.keras')
plt.figure(figsize=(12, 6))


plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()


plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

